In [44]:
import os, tarfile
import pyBigWig
import pybedtools
import zipfile, gzip
from gtfparse import read_gtf
import pandas as pd
pd.set_option('display.max_columns', None)
from Bio import SeqIO
import pandas as pd
import re

In [2]:
gtf_file_path = "/data/projects/Resources/Gencode_genome_annotation/gencode.v40.annotation.gtf"

In [3]:
def parse_attributes(attribute_string):
    # Split each attribute into key and value, then strip quotes and spaces
    return dict(item.strip().replace('"', '').split(' ') for item in attribute_string if item)

In [4]:
column_names = [
    "seqname", "source", "feature", "start", "end",
    "score", "strand", "frame", "attribute"
]

# Read the GTF file
gtf_df = pd.read_csv(gtf_file_path, sep="\t", comment='#', header=None, names=column_names)

In [5]:
# Step 1: Split the 'attribute' string into a list of strings for each key-value pair
attributes_list = gtf_df['attribute'].str.split(';')


# Apply the function to each row's attribute list
attributes_dicts = attributes_list.apply(parse_attributes)

# Step 3: Convert the list of dictionaries into a DataFrame
attributes_df = pd.DataFrame(list(attributes_dicts))

# Step 4: Combine the new attributes DataFrame with the original gtf_df
# This step assumes that the indexes are aligned and can be directly concatenated
combined_df = pd.concat([gtf_df, attributes_df], axis=1)

# Optionally, you can drop the original 'attribute' column if it's no longer needed
combined_df = combined_df.drop('attribute', axis=1)

In [6]:
combined_df

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,chr1,HAVANA,gene,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,11869,12227,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,1,ENSE00002234944.1,NaN,NaN,NaN
3,chr1,HAVANA,exon,12613,12721,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,2,ENSE00003582793.1,NaN,NaN,NaN
4,chr1,HAVANA,exon,13221,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,3,ENSE00002312635.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283857,chrM,ENSEMBL,transcript,15888,15953,.,+,.,ENSG00000210195.2,Mt_tRNA,MT-TT,3,HGNC:7499,NaN,ENST00000387460.2,Mt_tRNA,MT-TT-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN
3283858,chrM,ENSEMBL,exon,15888,15953,.,+,.,ENSG00000210195.2,Mt_tRNA,MT-TT,3,HGNC:7499,NaN,ENST00000387460.2,Mt_tRNA,MT-TT-201,NA,Ensembl_canonical,NaN,1,ENSE00001544475.2,NaN,NaN,NaN
3283859,chrM,ENSEMBL,gene,15956,16023,.,-,.,ENSG00000210196.2,Mt_tRNA,MT-TP,3,HGNC:7494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3283860,chrM,ENSEMBL,transcript,15956,16023,.,-,.,ENSG00000210196.2,Mt_tRNA,MT-TP,3,HGNC:7494,NaN,ENST00000387461.2,Mt_tRNA,MT-TP-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
combined_df.groupby("transcript_type").size()

transcript_type
IG_C_gene                                 267
IG_C_pseudogene                            24
IG_D_gene                                 115
IG_J_gene                                  58
IG_J_pseudogene                             6
IG_V_gene                                 989
IG_V_pseudogene                           474
IG_pseudogene                               2
Mt_rRNA                                     4
Mt_tRNA                                    44
TEC                                      2303
TR_C_gene                                  64
TR_D_gene                                  12
TR_J_gene                                 237
TR_J_pseudogene                             8
TR_V_gene                                 717
TR_V_pseudogene                            90
lncRNA                                 237669
miRNA                                    3758
misc_RNA                                 4424
non_stop_decay                           1657
nonsense_mediated_

In [8]:
combined_df.to_csv("/data/projects/Resources/Gencode_genome_annotation/gencode_annotation_V40.tsv", sep= "\t", index=False)

In [9]:
# Filter rows where 'feature' is 'transcript' and 'seqname' is not 'chrM'
gtf_transcript = combined_df[(combined_df["feature"] == "transcript") & (combined_df["seqname"] != "chrM")]

# Display the resulting DataFrame
gtf_transcript

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
5,chr1,HAVANA,transcript,12010,13670,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201,NA,Ensembl_canonical,OTTHUMT00000002844.2,NaN,NaN,PGO:0000019,NaN,NaN
13,chr1,HAVANA,transcript,14404,29570,.,-,.,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,2,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201,NA,Ensembl_canonical,OTTHUMT00000002839.1,NaN,NaN,PGO:0000005,NaN,NaN
26,chr1,ENSEMBL,transcript,17369,17436,.,-,.,ENSG00000278267.1,miRNA,MIR6859-1,3,HGNC:50039,NaN,ENST00000619216.1,miRNA,MIR6859-1-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN
29,chr1,HAVANA,transcript,29554,31097,.,+,.,ENSG00000243485.5,lncRNA,MIR1302-2HG,2,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202,5,Ensembl_canonical,OTTHUMT00000002840.1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283698,chrY,HAVANA,transcript,57209306,57210051,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210,1,PAR,OTTHUMT00000058833.1,NaN,NaN,NaN,NaN,NaN
3283701,chrY,HAVANA,transcript,57209887,57212186,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215,2,PAR,OTTHUMT00000058827.1,NaN,NaN,NaN,NaN,NaN
3283704,chrY,HAVANA,transcript,57210344,57212074,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211,1,PAR,OTTHUMT00000058834.1,NaN,NaN,NaN,NaN,NaN
3283708,chrY,HAVANA,transcript,57210591,57212074,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205,2,PAR,OTTHUMT00000058835.1,NaN,NaN,NaN,NaN,NaN


In [25]:
def extract_tss_regions_from_df(tss_df):
    """
    Extracts TSS (+50 and -50 bp) regions from the GTF DataFrame.
    :param df: A pandas DataFrame containing GTF data.
    :return: A DataFrame with TSS regions.
    """

    # Calculate TSS based on strand
    tss_df['TSS'] = tss_df.apply(lambda x: x['start'] if x['strand'] == '+' else x['end'], axis=1)

    # Adjust TSS based on strand for +50 and -50 bp
    tss_df['start_adj'] = tss_df.apply(lambda x: x['TSS'] - 50 if x['strand'] == '+' else x['TSS'], axis=1)
    tss_df['end_adj'] = tss_df.apply(lambda x: x['TSS'] + 50 if x['strand'] == '+' else x['TSS'] + 100, axis=1)

    # Select and rename relevant columns
    tss_regions = tss_df[['seqname', 'TSS','start_adj', 'end_adj', 'strand', 'gene_id', 'gene_type', 'gene_name', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name']]
    tss_regions.columns = ['Chromosome', 'TSS', '50BP_Start', '50BP_End', 'Strand', 'GeneID', 'gene_type', 'gene_name', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name']

    return tss_regions

In [26]:
tss_regions_df = extract_tss_regions_from_df(gtf_transcript)

/tmp/ipykernel_103852/3899473439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tss_df['TSS'] = tss_df.apply(lambda x: x['start'] if x['strand'] == '+' else x['end'], axis=1)
/tmp/ipykernel_103852/3899473439.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tss_df['start_adj'] = tss_df.apply(lambda x: x['TSS'] - 50 if x['strand'] == '+' else x['TSS'], axis=1)
/tmp/ipykernel_103852/3899473439.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [27]:
tss_regions_df

,Chromosome,TSS,50BP_Start,50BP_End,Strand,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name
1,chr1,11869,11819,11919,+,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202
5,chr1,12010,11960,12060,+,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201
13,chr1,29570,29570,29670,-,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201
26,chr1,17436,17436,17536,-,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,NaN,ENST00000619216.1,miRNA,MIR6859-1-201
29,chr1,29554,29504,29604,+,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283698,chrY,57209306,57209256,57209356,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210
3283701,chrY,57209887,57209837,57209937,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215
3283704,chrY,57210344,57210294,57210394,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211
3283708,chrY,57210591,57210541,57210641,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205


In [28]:
# Define a custom aggregation function
def custom_agg(series):
    # If all values are the same, return any one of them; otherwise, return the list of unique values
    if series.nunique() == 1:
        return series.iloc[0]
    else:
        return list(series.unique())

In [30]:
# Group by the columns 'Chromosome', 'Start', 'End', 'Strand'
# and aggregate other columns
unique_tss_df = tss_regions_df.groupby(['Chromosome', '50BP_Start', '50BP_End', 'Strand']).agg(custom_agg).reset_index()

In [31]:
unique_tss_df

,Chromosome,50BP_Start,50BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name
0,chr1,11819,11919,+,11869,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202
1,chr1,11960,12060,+,12010,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201
2,chr1,17436,17536,-,17436,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,[nan],ENST00000619216.1,miRNA,MIR6859-1-201
3,chr1,29504,29604,+,29554,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202
4,chr1,29570,29670,-,29570,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215293,chrY,57209256,57209356,+,57209306,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210
215294,chrY,57209837,57209937,+,57209887,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215
215295,chrY,57210294,57210394,+,57210344,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211
215296,chrY,57210541,57210641,+,57210591,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205


In [39]:
def read_genome(fasta_file):
    """
    Reads a genome fasta file and returns a dictionary of chromosomes and sequences.
    :param fasta_file: Path to the genome fasta file.
    :return: A dictionary with chromosome names as keys and sequences as values.
    """
    genome = {}
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        genome[seq_record.id] = seq_record.seq
    return genome

def fetch_sequences_for_regions(df, genome):
    """
    Fetches sequences for regions specified in the DataFrame from the genome.
    :param df: DataFrame with the regions (must contain 'Chromosome', 'Start', 'End', and 'Strand' columns).
    :param genome: Dictionary of genome sequences.
    :return: List of sequences corresponding to the regions.
    """
    sequences = []
    for index, row in df.iterrows():
        # Extracting the sequence
        seq = genome[row['Chromosome']][row['50BP_Start']:row['50BP_End']]
        
        # Reverse complement if the gene is on the negative strand
        if row['Strand'] == '-':
            seq = seq.reverse_complement()
        seq= seq.upper()
        
        sequences.append(str(seq))  # Converts Seq object to string
    return sequences

In [40]:
# Paths
genome_path = "/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa"

In [41]:
# Load the genome and TSS regions
genome = read_genome(genome_path)

In [42]:
# Fetch sequences
sequences = fetch_sequences_for_regions(unique_tss_df, genome)

In [62]:
unique_tss_df['Sequence'] = sequences  # Add sequences to the DataFrame
unique_tss_df

,Chromosome,50BP_Start,50BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,Sequence,TATA_Box_Positions
0,chr1,11819,11919,+,11869,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,GTTTAAACGAGATTGCCAGCACCGGGTATCATTCACCATTTTTCTT...,[]
1,chr1,11960,12060,+,12010,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201,GGGCCTTTGAGAGGTCACAGGGTCTTGATGCTGTGGTCTTCATCTG...,[]
2,chr1,17436,17536,-,17436,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,[nan],ENST00000619216.1,miRNA,MIR6859-1-201,ATCAGACACAGGTACAGCACATAGGCCAGGAGCCAGGGGGTGACGG...,[]
3,chr1,29504,29604,+,29554,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202,CGAGTCGCGGGCCTGGGCACGGAACTCACGCTCACTCCGAGCTCCC...,[]
4,chr1,29570,29670,-,29570,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201,GAGGGGAAGGCAGCGATGGGACAAAGGACGGAGGTCTAGGAAGAGG...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215293,chrY,57209256,57209356,+,57209306,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210,CTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAGGAGAA...,[]
215294,chrY,57209837,57209937,+,57209887,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215,ACTAACACCACCCCCACCGCCCCCACCACCACCCCCAGCTCCTGAG...,[]
215295,chrY,57210294,57210394,+,57210344,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211,CAGTGTAACCCTAGCATCCAGAAGTGGCACAAAACCCCTCTGCTGG...,[]
215296,chrY,57210541,57210641,+,57210591,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205,TGACTTCCCTCCCAGCAGGCTCTTGGACACAGTAAGCTTCCCCAGC...,[]


In [71]:
# Your existing function
def find_tata_box(sequence):
    # tata_pattern_plus = r"TAT[AT][AT][AG]"  # Simplified TATA box pattern
    # tata_pattern_minus= r"[TC][AT][AT]ATA"
    tata_pattern = r"TAT[AT][AT][AG]|[TC][AT][AT]ATA"
    for match in re.finditer(tata_pattern, sequence):
        yield match.start()

In [72]:
# Assuming 'unique_tss_df' is your DataFrame with the sequences

# Apply the function to each sequence and store the results in a new column
unique_tss_df['TATA_Box_Positions'] = unique_tss_df['Sequence'].apply(lambda seq: list(find_tata_box(seq)))

In [75]:
# Create the three dataframes based on the conditions
# 1. TATA Core Promoter DataFrame
tata_core_df = unique_tss_df[unique_tss_df['TATA_Box_Positions'].apply(
    lambda positions: any(20 <= pos <= 35 for pos in positions))]

# 2. Sequences containing TATA Box but not functioning as TATA core promoter
tata_non_functional_df = unique_tss_df[unique_tss_df['TATA_Box_Positions'].apply(
    lambda positions: any(pos not in range(20, 36) for pos in positions) if positions else False)]

# 3. DataFrame without TATA Box
no_tata_df = unique_tss_df[unique_tss_df['TATA_Box_Positions'].apply(len) == 0]

In [76]:
tata_core_df

,Chromosome,50BP_Start,50BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,Sequence,TATA_Box_Positions
5,chr1,30217,30317,+,30267,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000469289.1,lncRNA,MIR1302-2HG-201,AAAGAAAGAAAAGAGGCTTTTTTAAAAAGTTAAATATATAAAAAGA...,"[30, 36]"
14,chr1,120932,121032,-,120932,ENSG00000238009.6,lncRNA,ENSG00000238009,[nan],OTTHUMG00000001096.2,ENST00000466430.5,lncRNA,ENST00000466430,AATAGATATACAATATATAATATGTAATAGATATATATTATATATA...,"[13, 31, 38, 51, 58]"
25,chr1,157887,157987,-,157887,ENSG00000222623.1,snRNA,RNU6-1100P,HGNC:48063,[nan],ENST00000410691.1,snRNA,RNU6-1100P-201,TATTTACTGAGCTGTTACTAGGTGCCTATAAATAGCATAGATTTTT...,"[0, 26]"
31,chr1,182646,182746,+,182696,ENSG00000279928.2,unprocessed_pseudogene,DDX11L17,HGNC:55080,OTTHUMG00000191962.1,ENST00000624431.2,unprocessed_pseudogene,DDX11L17-201,CCCCTGTTGTCTGCATGTAACTTAATACCACAACCAGGCATAGGGG...,[21]
38,chr1,297502,297602,-,297502,ENSG00000228463.10,transcribed_processed_pseudogene,ENSG00000228463,[nan],OTTHUMG00000002552.3,ENST00000424587.7,processed_transcript,ENST00000424587,ATGTAATAGATATACAATATATAATATATAATAGATATATAATATT...,"[17, 24, 35, 45, 58]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215174,chrY,24501505,24501605,-,24501505,ENSG00000232003.2,processed_pseudogene,ZNF736P12Y,HGNC:38431,OTTHUMG00000045107.1,ENST00000443152.1,processed_pseudogene,ZNF736P12Y-201,TTGTCACCTTGTTAATGTGCTTTGTGTAACTAATATTTGTAAAATG...,[29]
215182,chrY,24683607,24683707,+,24683657,ENSG00000226270.3,processed_pseudogene,ZNF736P2Y,HGNC:37727,OTTHUMG00000045110.1,ENST00000416040.1,processed_pseudogene,ZNF736P2Y-201,TGAGTCTAATGAGGGAGAGATTTATAGTCCTTTTATTTTCTTAGCT...,[20]
215195,chrY,24978317,24978417,-,24978317,ENSG00000278602.1,unprocessed_pseudogene,ENSG00000278602,[nan],OTTHUMG00000188109.1,ENST00000616864.1,unprocessed_pseudogene,ENST00000616864,AGAAATTTAAAATATCTATAGTAAATAAATGACATTAATTCTGCAT...,[21]
215241,chrY,25904468,25904568,+,25904518,ENSG00000232914.1,unprocessed_pseudogene,TRAPPC2P4,HGNC:10713,OTTHUMG00000045261.1,ENST00000421750.1,unprocessed_pseudogene,TRAPPC2P4-201,AAAAATATGCACTATTTTTTGCTTCATTAGGAGCCATATATTGAAG...,[34]


In [121]:
tata_core_df.to_csv("/data/private/pdutta/DNABERT_data/Core_Prom_new/TATA/raw_positive.tsv", sep="\t", index=False)

In [114]:
# Your existing function
def find_tata_box_stringent(sequence):
    # tata_pattern_plus = r"TAT[AT][AT][AG]"  # Simplified TATA box pattern
    # tata_pattern_minus= r"[TC][AT][AT]ATA"
    tata_pattern = r"TATA[AT][AT][AG]"
    for match in re.finditer(tata_pattern, sequence):
        yield match.start()

In [116]:
# Initialize a list to hold results
results = []
#[f'chr{i}' for i in range(1, 23)] +
# Iterate through specified chromosomes only
for chrom in [f'chr{i}' for i in range(1, 23)]+ ['chrX', 'chrY']:
    sequence = genome[chrom]  # Fetch the sequence for the chromosome
    length = len(sequence)
    print(chrom)
    #print(sequence[0:10])

    # Break down the chromosome into 100 bp segments and search
    for i in range(0, length - 100 + 1, 50):  # 50 bp step for some overlap; adjust as needed
        segment = str(sequence[i:i+100]).upper()  # Ensure segment is a string and uppercase
        #print(segment)
        # Check if segment is a proper string before processing
        if isinstance(segment, str):
            for pos in find_tata_box_stringent(segment):
                if 75 <= pos <= 81:
                    results.append({
                        'Chromosome': chrom,
                        'Start': i,
                        'End': i + 100,
                        'TATA_Box_Position': pos,
                        'Sequence': segment
                    })
        else:
            print(f"Non-string segment detected at {chrom}:{i}-{i+100}")

# Convert results into a DataFrame
tata_box_df = pd.DataFrame(results)

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY


In [117]:
tata_box_df

,Chromosome,Start,End,TATA_Box_Position,Sequence
0,chr1,24300,24400,81,TCTATTGCTTATTTAGGTCTTTCTCTCCTGCCAGAATGTGCGCTCC...
1,chr1,27350,27450,79,ATTGAGATGTGCCATAAGTGTAAAATATGCACCAAATTTCAAAGGC...
2,chr1,43550,43650,78,CCCACAGCTAACATCATAATAGTAAAATCATCACACTGGTAAAAAA...
3,chr1,45650,45750,75,ATGTGTTAATTAACTTGATTTATTCATTCAACAACATACACATATA...
4,chr1,50700,50800,80,AAAATGCCATTATCCAAAGTGGTTCTCTAGAGATGCTGATGTATAT...
...,...,...,...,...,...
459571,chrY,57150100,57150200,79,ACCCCCTTATCTGGCTGTTCATGTGTTTTCTTTACCATGTCCTTTA...
459572,chrY,57151700,57151800,81,TGGGGGGGGCAGTCTGGTGTTACTGAGCCCTTAAACTATTAGGTAT...
459573,chrY,57171600,57171700,76,ACTGACCTTATATTGCTAGTCTTTGCTTTTGTACCTCTTTAGGGCA...
459574,chrY,57174800,57174900,77,ATACCCGTGAGGAAAATGAAGGACTAGCACATTGGGCCTTAATAAA...


In [118]:
# Convert DataFrames to BED format for pybedtools
tata_box_bed = pybedtools.BedTool.from_dataframe(tata_box_df[['Chromosome', 'Start', 'End']])
tata_core_bed = pybedtools.BedTool.from_dataframe(tata_core_df[['Chromosome', '50BP_Start', '50BP_End']])
# Use the intersect function with the -v option to get non-intersecting regions
non_intersecting = tata_box_bed.intersect(tata_core_bed, v=True)

In [119]:
# Convert back to DataFrame
non_intersecting_df = non_intersecting.to_dataframe(names=['Chromosome', 'Start', 'End'])

# You might want to merge back additional information from tata_box_df if needed
tata_box_df_filtered = pd.merge(non_intersecting_df, tata_box_df, on=['Chromosome', 'Start', 'End'])

In [120]:
tata_box_df_filtered

,Chromosome,Start,End,TATA_Box_Position,Sequence
0,chr1,24300,24400,81,TCTATTGCTTATTTAGGTCTTTCTCTCCTGCCAGAATGTGCGCTCC...
1,chr1,27350,27450,79,ATTGAGATGTGCCATAAGTGTAAAATATGCACCAAATTTCAAAGGC...
2,chr1,43550,43650,78,CCCACAGCTAACATCATAATAGTAAAATCATCACACTGGTAAAAAA...
3,chr1,45650,45750,75,ATGTGTTAATTAACTTGATTTATTCATTCAACAACATACACATATA...
4,chr1,50700,50800,80,AAAATGCCATTATCCAAAGTGGTTCTCTAGAGATGCTGATGTATAT...
...,...,...,...,...,...
459193,chrY,57150100,57150200,79,ACCCCCTTATCTGGCTGTTCATGTGTTTTCTTTACCATGTCCTTTA...
459194,chrY,57151700,57151800,81,TGGGGGGGGCAGTCTGGTGTTACTGAGCCCTTAAACTATTAGGTAT...
459195,chrY,57171600,57171700,76,ACTGACCTTATATTGCTAGTCTTTGCTTTTGTACCTCTTTAGGGCA...
459196,chrY,57174800,57174900,77,ATACCCGTGAGGAAAATGAAGGACTAGCACATTGGGCCTTAATAAA...


In [122]:
tata_box_df_filtered.to_csv("/data/private/pdutta/DNABERT_data/Core_Prom_new/TATA/raw_negative_all.tsv", sep="\t", index=False)

In [125]:
tata_box_df_filtered_sample = tata_box_df_filtered.sample(n=8500, random_state=1).reset_index(drop=True)

In [126]:
tata_box_df_filtered_sample

,Chromosome,Start,End,TATA_Box_Position,Sequence
0,chr5,26073800,26073900,76,GACTATACCAAGGCATTTCTACACCTTTCTTTATCATCTGTACATT...
1,chr16,7125250,7125350,75,GTTGTCGTTCAGCCTTTCTAACCTGCAGCATCCTCATTTGTGCAGT...
2,chr8,107362200,107362300,80,TAACAGGCAACGTGCCATGAATACATAAATATGAAAAAGATGCAGT...
3,chr10,123781250,123781350,79,GAGGCAATGTCTGCTCTTCACTTGATTCCCTTCCATATAAATTGAC...
4,chr3,149746400,149746500,78,AGGAGTAGTTTCAGTAGAGAGGGGGTGTATTAGTCCTTTCTCACAC...
...,...,...,...,...,...
8495,chr3,105119050,105119150,79,ACACATAAATATAGTCATCCACTGCCTACGGAAATTTTAGTCAATG...
8496,chr10,10642300,10642400,76,GTCACACCAACCTTCATACTCTCTCTGAATCACTTCTCCAGCACAG...
8497,chr18,1045150,1045250,80,ATACATATATGTAGATTCCATCATATATGGGAACAAAAGTTTCCAT...
8498,chr6,51491650,51491750,79,ACAACATGGGAGAAAGGCAGGCAGAAGCAAGATACCCCATCCACTT...


In [127]:
tata_box_df_filtered_sample.to_csv("/data/private/pdutta/DNABERT_data/Core_Prom_new/TATA/raw_negative.tsv", sep="\t", index=False)

In [8]:
df_pos

,Sequence
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...
...,...
4320,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...
4321,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...
4322,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...
4323,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...


In [9]:
df_pos['Label']=1

In [10]:
df_pos = df_pos.reset_index(drop=True)
df_pos

,Sequence,Label
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...,1
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...,1
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...,1
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...,1
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...,1
...,...,...
236970,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...,1
236971,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1
236972,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...,1
236973,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1


In [12]:
df_pos.to_csv("/data/projects/DNABERT_data/Core_promoters/Core_promoter_regions/positive_set.tsv", sep="\t", index=False)